In [131]:
import urllib
import os
import pandas as pd

from bs4 import BeautifulSoup
from termcolor import colored

In [54]:
Z_MIN = 1
Z_MAX = 112

In [4]:
def download_isotopes_list(Z):
    request_url = f"http://nucleardata.nuclear.lu.se/toi/listnuc.asp?sql=&Z={Z}"
    urllib.request.urlretrieve(request_url, f"downloads/find_isotopes/z_{Z}.html")

In [5]:
def download_range(z_min, z_max):
    for Z in range(z_min, z_max + 1):
        download_isotopes_list(Z)

In [6]:
download_range(Z_MIN, Z_MAX)

In [62]:
def parse_isotopes_one(Z):
    html_path = f"downloads/find_isotopes/z_{Z}.html"
    isotopes_lst_html = open(html_path, "r")
    bs = BeautifulSoup(isotopes_lst_html.read(), 'lxml')
    table = bs.find_all("table")[0]
    nuclide_lst = table.find_all('th')[9:]
    abbr = str(nuclide_lst[0].find('a')).split('</sup>')[1][:-4]
    out_file_path = f"downloads/find_isotopes_parsed/{Z}_{abbr}.txt"
    out_file = open(out_file_path, "w")
    out_file.write(f"{Z}\n")
    out_file.write(f"{abbr}\n")
    for nuclide in nuclide_lst:
        out_file.write(f"{nuclide.find('sup').get_text()}\n")

    out_file.close()

In [63]:
def parse_isotopes_range(z_min, z_max):
    for Z in range(z_min, z_max + 1):
        parse_isotopes_one(Z)

In [65]:
parse_isotopes_range(Z_MIN,Z_MAX)

In [116]:
def download_all_isotopes(Z):
    down_dir = "downloads/find_isotopes_parsed"
    files_lst = os.listdir(down_dir)
    file_name = [s for s in files_lst if str(Z) == s.split('_')[0]]
    isotopes_lst_file = open(f"{down_dir}/{file_name[0]}", "r")
    lines = isotopes_lst_file.readlines()
    abbr = lines[1].strip()
    A_lst = lines[2:]
    for A in A_lst:
        A = int(A)
        if A < 10:
            str_A = '00' + str(A)
        elif A < 100:
            str_A = '0' + str(A)
        url = f"http://nucleardata.nuclear.lu.se/toi/nuclide.asp?iZA={Z}0{str_A}"
        urllib.request.urlretrieve(url, f"downloads/isotopes_html/{A}_{abbr}_{Z}.html")

In [127]:
def extract_Igamma(A, element, Z):
    html_file = open(f"downloads/isotopes_html/{A}_{element}_{Z}.html", "r")
    soup = BeautifulSoup(html_file.read(), 'lxml')
    
    try:
        gammas_table = soup.find_all("table")[4]
        gammas_rows = gammas_table.find_all('tr')[3:-1]
    except:
        A = int(A)
        if A < 10:
            str_A = '00' + str(A)
        elif A < 100:
            str_A = '0' + str(A)
        print(colored(f"Seems like there are no gamma-lines known for isotope {A}{element}.", 'red'))
        print(colored("Check yellow pages for reference.", 'yellow'))
        print(colored(f"http://nucleardata.nuclear.lu.se/toi/nuclide.asp?iZA={Z}0{str_A}", 'yellow'))
        return 1
    energy = []
    sigm_energy = []
    i = []
    sigm_i = []

    for row in gammas_rows:
        cells = row.find_all('td')
        
        e_val = cells[0].get_text(strip=True)
        i_val = cells[1].get_text(strip=True)
        try:
            ig_val = float(i_val[:-1])
            sigm_ig_val = float(i_val[-1])
        except:
            ig_val = float('NaN')
            sigm_ig_val = float('NaN')
       
        energy.append(float(e_val[:-1]))
        sigm_energy.append(int(e_val[-1]))
        i.append(ig_val)
        
        sigm_i.append(sigm_ig_val)


    df_dict = {
        "E_tab": energy,
        "sigm_E": sigm_energy, 
        "Ig": i,
        "sigm_Ig": sigm_i
        }
    df = pd.DataFrame(df_dict)
    df_name = f'downloads/ig_db/{A}{element}.csv'
    df.to_csv(df_name)
   
    print(f"Ig extracted from file 'downloads/{A}{element}.html' into '{df_name}'.")
    return 0

In [132]:
extract_Igamma(20, "F", 9)

Ig extracted from file 'downloads/20F.html' into 'downloads/ig_db/20F.csv'.


0

In [120]:
for i in range(1,10):
    download_all_isotopes(i)

downloads/isotopes_html/1_H.html
downloads/isotopes_html/2_H.html
downloads/isotopes_html/3_H.html
downloads/isotopes_html/4_H.html
downloads/isotopes_html/5_H.html
downloads/isotopes_html/6_H.html
downloads/isotopes_html/3_He.html
downloads/isotopes_html/4_He.html
downloads/isotopes_html/5_He.html
downloads/isotopes_html/6_He.html
downloads/isotopes_html/7_He.html
downloads/isotopes_html/8_He.html
downloads/isotopes_html/9_He.html
downloads/isotopes_html/10_He.html
downloads/isotopes_html/4_Li.html
downloads/isotopes_html/5_Li.html
downloads/isotopes_html/6_Li.html
downloads/isotopes_html/7_Li.html
downloads/isotopes_html/8_Li.html
downloads/isotopes_html/9_Li.html
downloads/isotopes_html/10_Li.html
downloads/isotopes_html/11_Li.html
downloads/isotopes_html/12_Li.html
downloads/isotopes_html/5_Be.html
downloads/isotopes_html/6_Be.html
downloads/isotopes_html/7_Be.html
downloads/isotopes_html/8_Be.html
downloads/isotopes_html/9_Be.html
downloads/isotopes_html/10_Be.html
downloads/isoto

In [121]:
lst = os.listdir("downloads/isotopes_html")

In [122]:
len(lst)

107

In [96]:
matching = [s for s in lst if "12" == s.split('_')[0]]

In [97]:
matching[0]

'12_Mg.txt'